<a href="https://colab.research.google.com/github/BorutFlis/Scripts/blob/main/keras_custom_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import datetime
import re
from time import time
from sklearn.decomposition import PCA
import numpy as np
start = time()
dogs=pd.read_csv("Dogs.csv",parse_dates=[["Date","Time"]], dayfirst=True)
races=pd.read_csv("bsphistory.csv",parse_dates=[["Date","Time"]], dayfirst=True)
dogs = dogs.loc[:,~dogs.columns.duplicated()]
races = races.loc[:,~races.columns.duplicated()]
#dogs["Sire"]=dogs["Sire"].map(lambda x: re.sub(r'\W+', '', x))
#re.sub('[\W_]+', '', dogs["Sire"])
#dogs = dogs.rename(columns = lambda x:re.sub('[^0-9a-zA-Z]', '', x))
#races = races.rename(columns = lambda x:re.sub('[^0-9a-zA-Z]', '', x))
dogs.reset_index()
#dogs=dogs.fillna(0)
#races=races.fillna(0)
dogs=dogs.drop(['Win/Sec'],axis=1)
dogs=dogs.drop(['Split'],axis=1)
dogs=dogs.drop(['Gng'],axis=1)

In [5]:
dass=dogs["Remarks"]
#dummies = pd.get_dummies(dass.str.findall("([A-Z][A-z]+)").explode()).sum(level = 0)
dummies = pd.get_dummies(dogs["Remarks"].str.findall("([A-Z][a-z]+)").explode()).sum(level = 0)

In [6]:

from sklearn.preprocessing import StandardScaler
dummies = StandardScaler().fit_transform(dummies)
pca = PCA(8)
pca.fit(dummies)
dummies=pd.DataFrame(pca.transform(dummies))
print(dummies)
#dogs = pd.concat([dogs,dummies], axis=1)

              0         1         2  ...         5         6         7
0     -0.010554  0.021075  0.560146  ... -0.219482 -0.100998  0.191476
1     -0.015178  0.003312  0.816210  ... -0.490458 -0.173191 -0.035825
2     -0.006645  0.006850  0.077511  ... -0.011124 -0.013387  0.020985
3     -0.002707  0.039737  0.177575  ... -0.095553  0.040437 -0.032227
4     -0.006645  0.006850  0.077511  ... -0.011124 -0.013387  0.020985
...         ...       ...       ...  ...       ...       ...       ...
85390 -0.001385  0.015139  0.103022  ... -0.045835  0.004603  0.007316
85391 -0.006814  0.009599  0.027697  ...  0.008238  0.003565  0.068937
85392 -0.013757 -0.047212  0.534807  ...  0.886086  0.156245  0.553624
85393 -0.016119 -0.008664 -0.630109  ...  0.161084 -0.049303 -0.050621
85394 -0.012074  0.001309  0.002186  ...  0.042949 -0.014426  0.082605

[85395 rows x 8 columns]


In [7]:
dogs["A0"]=dummies[0]
dogs["A1"]=dummies[1]
dogs["A2"]=dummies[2]
dogs["A3"]=dummies[3]
dogs["A4"]=dummies[4]
dogs["A5"]=dummies[5]
dogs["A6"]=dummies[6]
dogs["A7"]=dummies[7]
#dogs["A8"]=dummies[8]
#dogs["A9"]=dummies[9]

In [8]:
start = time()
#data cleaning

nan_value = float("NaN")
dogs.Sire.replace("", nan_value, inplace=True)
dogs.Sire.dropna(inplace=True)
#dogs=dogs.dropna()
#races=races.dropna()
dogs.drop(dogs[dogs.Fin.eq("NR")].index,inplace=True)
dogs.drop(dogs[dogs.Sire.eq(" ")].index,inplace=True)
dogs.Sire = dogs.Sire.dropna(how='any',axis=0) 
dogs.drop(dogs[dogs.Fin.eq("N")].index,inplace=True)
dogs.drop(dogs[dogs.CalTm.eq(0)].index,inplace=True)
dogs.drop(dogs[dogs.Fin.eq("n")].index,inplace=True)
dogs.drop(dogs[dogs.Bends.isna()].index,inplace=True)
#dogs.drop(dogs[dogs["Win/Sec"].eq("No race")].index,inplace=True)
#dogs.loc[dogs["Gng"].eq("N"),"Gng"]=0
dogs.loc[dogs["By"].eq("shd"),"By"]=0.45
dogs.loc[dogs["By"].eq("hd"),"By"]=0.15
dogs.loc[dogs["By"].eq("nk"),"By"]=0.25
dogs.loc[dogs["By"].eq("DNF"),"By"]=6
dogs.loc[dogs["By"].eq("dist"),"By"]=0
dogs.loc[dogs["By"].eq("dht"),"By"]=0
#dogs.loc[dogs["Gng"].eq("N"""),"Gng"]=0
races.drop(races[races.Position.eq("N")].index, inplace=True)
races.drop(races[races.Position.eq("xx")].index, inplace=True)

dogs["Fin"].replace("=",1,inplace=True)
dogs.dropna(subset=["Fin"],inplace=True)
dogs = dogs.astype({"Fin":int})
#dogs=dogs.dropna()
#races=races.dropna()
dogs.sort_values(by="Date_Time",inplace=True)
dogs.reset_index(drop=True, inplace=True)

dogs["Fin_binary"]=dogs["Fin"].apply(lambda x: 1 if x<3 else 0)
dogs["won"]=dogs["Fin"].apply(lambda x: 1 if x==1 else 0)

c=dogs.groupby(by="Dog name")[["SP","Fin_binary","won"]].agg({"won":["cumsum"],"SP":["cumsum","cumcount"], "Fin_binary":"cumsum"})
end = time()
running_time = end - start
print('1 time cost: %.5f sec' %running_time)
c[("SP","cumcount")]+=1
dogs.insert(len(dogs.columns),"Total races",c[("SP","cumcount")])
dogs.insert(len(dogs.columns),"Average SP", c[("SP","cumsum")]/c[("SP","cumcount")])
dogs.insert(len(dogs.columns),"Total poss", c[("Fin_binary","cumsum")])
dogs.insert(len(dogs.columns),"Total win", c[("won","cumsum")])
Total_win7 = dogs.groupby(by="Dog name")["won"].transform(lambda x: x.rolling(7, 1).sum())
dogs.insert(len(dogs.columns),"Total win7", Total_win7)


def convo_mean(df, grp_by, attr, window_size=None):
    grouped= df.groupby(by=grp_by)[attr]
    s = [[], []]
    for k, group in grouped:
        if window_size != None:
            convolved_SP = np.convolve(group, np.ones(window_size))
            convolved_SP = convolved_SP[:-(window_size - 1)]
            divisor = np.ones(len(convolved_SP)) * window_size
            divisor[:min(len(divisor), window_size-1)] = np.arange(1, min(len(divisor) + 1, window_size))
        else:
            convolved_SP = np.convolve(group, np.ones(len(group)))
            convolved_SP = convolved_SP[:len(group)]
            divisor=np.arange(1,len(group)+1)
        convolved_SP = convolved_SP / divisor
        s[0].extend(group.index)
        s[1].extend(convolved_SP)
    return_s = pd.Series(s[1], index=s[0])
    return_s.sort_index(inplace=True)
    return return_s

Average_SP7 = convo_mean(dogs,"Dog name","SP",7)
dogs.insert(len(dogs.columns),"Average SP7", Average_SP7)
Average_SP15 = convo_mean(dogs,"Dog name","SP",15)
dogs.insert(len(dogs.columns),"Average SP15", Average_SP15)
Average_SP3 = convo_mean(dogs,"Dog name","SP",3)
dogs.insert(len(dogs.columns),"Average SP3", Average_SP3)


StdDevFin = dogs.groupby(by="Dog name")["Fin"].transform(lambda x: x.expanding().std())
StdDevFin7 = dogs.groupby(by="Dog name")["Fin"].transform(lambda x: x.rolling(7,1).std())
dogs.insert(len(dogs.columns),"StdDevFin", StdDevFin)
dogs.insert(len(dogs.columns),"StdDevFin7", StdDevFin7)

Place_Average_fin = convo_mean(dogs,"Dog name", "Fin_binary")
Place_Average_fin7 = convo_mean(dogs,"Dog name", "Fin_binary", 7)
Place_Average_fin3 = convo_mean(dogs, "Dog name","Fin_binary",3)
dogs.insert(len(dogs.columns),"PlaceAverageFin", Place_Average_fin)
dogs.insert(len(dogs.columns),"PlaceAverageFin7", Place_Average_fin7)
dogs.insert(len(dogs.columns),"PlaceAverageFin3", Place_Average_fin3)
end = time()
running_time = end - start
print('2time cost: %.5f sec' %running_time)
def get_avg_bends(x):
    l=[int(char) for char in x if char.isdigit()]
    if len(l)>0:
        return sum(l)/len(l)

dogs = dogs.astype({"Bends":str})
dogs["AverageBends"] = dogs["Bends"].apply(lambda x: get_avg_bends(x))
Average_bends = convo_mean(dogs,"Dog name", "AverageBends")
dogs.insert(len(dogs.columns),"AverageBends_cum", Average_bends)

dogs["AverageBends_first2"] = dogs["Bends"].apply(lambda x: get_avg_bends(x[:2]))
Average_bends = convo_mean(dogs,"Dog name", "AverageBends_first2")
dogs.insert(len(dogs.columns),"AverageBends_cum_first2", Average_bends)

dogs["AverageBends_last2"] = dogs["Bends"].apply(lambda x: get_avg_bends(x[2:]))
Average_bends = convo_mean(dogs,"Dog name", "AverageBends_last2")
dogs.insert(len(dogs.columns),"AverageBends_cum_last2", Average_bends)

dogs["profit"] = dogs.apply(lambda x: x["SP"] if x["won"] else -1,axis=1)
dogs["distwon"] = dogs.apply(lambda x: 0 if x["won"] else x["By"],axis=1)
dogs["distwon2"] = dogs.apply(lambda x: x["By"] if x["won"] else x["By"],axis=1)
Average_profit = convo_mean(dogs, "Dog name", "profit")
dogs.insert(len(dogs.columns),"Dogs profit", Average_profit)
Average_dist = dogs.groupby(by="Dog name")["distwon"].transform(lambda x: x.expanding().mean())
dogs.insert(len(dogs.columns),"AverageDistWonLoss", Average_dist)
Average_dist2 = dogs.groupby(by="Dog name")["distwon2"].transform(lambda x: x.expanding().mean())
dogs.insert(len(dogs.columns),"AverageDistWonLoss2", Average_dist)

end = time()
running_time = end - start
print('3time cost: %.5f sec' %running_time)
dogs.Sire=dogs.Sire.str.upper()
dogs.Dame=dogs.Dame.str.upper()
dogs["SireProfitability"] = convo_mean(dogs,"Sire", "profit")
dogs["DameProfitability"] = convo_mean(dogs,"Dame","profit")

dogs["Wins on this distance"] = convo_mean(dogs,["Dog name", "Distance"],"won")

dogs["Average position on this distance"] = convo_mean(dogs, ["Dog name","Distance"], "Fin")
dogs["Sire position on this distance"] = convo_mean(dogs, ["Dog name","Sire"], "Fin") #not really on this distance but anyway
dogs["Dame position on this distance"] = convo_mean(dogs, ["Dog name","Dame"], "Fin") #not really on this distance but anyway

dogs["GradeWinsAverage"] = convo_mean(dogs, ["Dog name","Distance","Grade"], "won")
dogs["GradeProfitability"] = convo_mean(dogs, ["Dog name","Distance","Grade"], "profit")
end = time()
running_time = end - start
print('4time cost: %.5f sec' %running_time)
dogs["Lowest CalTm"] = dogs.groupby(by=["Dog name","Distance"])["CalTm"].transform(lambda x: x.expanding().min()) #need to be average ikke min
dogs["Lowest CalTm Av"] = convo_mean(dogs, ["Dog name","Distance"], "CalTm")
dogs["Lowest CalTm Av Sire"] = convo_mean(dogs, ["Dog name","Sire"], "CalTm")
dogs["Lowest CalTm Av Dame"] = convo_mean(dogs, ["Dog name","Dame"], "CalTm")

dogs["Midcount"] = dogs["Remarks"].apply(lambda x: len(re.findall("mid",str(x),re.IGNORECASE)))
dogs["Midcount"] = dogs.groupby(by="Dog name")["Midcount"].cumsum()
dogs["EvCh"] = dogs["Remarks"].apply(lambda x: len(re.findall("Evch",str(x),re.IGNORECASE)))
dogs["EvCh"] = dogs.groupby(by="Dog name")["EvCh"].cumsum()
dogs["Bmp"] = dogs["Remarks"].apply(lambda x: len(re.findall("Bmp",str(x),re.IGNORECASE)))
dogs["Bmp"] = dogs.groupby(by="Dog name")["Bmp"].cumsum()
dogs["Crd"] = dogs["Remarks"].apply(lambda x: len(re.findall("Crd",str(x),re.IGNORECASE)))
dogs["Crd"] = dogs.groupby(by="Dog name")["Crd"].cumsum()
dogs["SAw"] = dogs["Remarks"].apply(lambda x: len(re.findall("SAw",str(x),re.IGNORECASE)))
dogs["SAw"] = dogs.groupby(by="Dog name")["SAw"].cumsum()
dogs["QAw"] = dogs["Remarks"].apply(lambda x: len(re.findall("QAw",str(x),re.IGNORECASE)))
dogs["QAw"] = dogs.groupby(by="Dog name")["QAw"].cumsum()
dogs["Ld1"] = dogs["Remarks"].apply(lambda x: len(re.findall("Ld1",str(x),re.IGNORECASE)))
dogs["Ld1"] = dogs.groupby(by="Dog name")["Ld1"].cumsum()
dogs["QAwALd"] = dogs["Remarks"].apply(lambda x: len(re.findall("QAw,ALd",str(x),re.IGNORECASE)))
dogs["QAwALd"] = dogs.groupby(by="Dog name")["QAwALd"].cumsum()
dogs["EPLd1"] = dogs["Remarks"].apply(lambda x: len(re.findall("EP,Ld1",str(x),re.IGNORECASE)))
dogs["EPLd1"] = dogs.groupby(by="Dog name")["EPLd1"].cumsum()
dogs["LdRnln"] = dogs["Remarks"].apply(lambda x: len(re.findall("LdRnln",str(x),re.IGNORECASE)))
dogs["LdRnln"] = dogs.groupby(by="Dog name")["LdRnln"].cumsum()
dogs["EPALd"] = dogs["Remarks"].apply(lambda x: len(re.findall("EP,ALd",str(x),re.IGNORECASE)))
dogs["EPALd"] = dogs.groupby(by="Dog name")["EPALd"].cumsum()
dogs["ClrRn"] = dogs["Remarks"].apply(lambda x: len(re.findall("ClrRn",str(x),re.IGNORECASE)))
dogs["ClrRn"] = dogs.groupby(by="Dog name")["ClrRn"].cumsum()
dogs["EvAwRnOn"] = dogs["Remarks"].apply(lambda x: len(re.findall("EvAw,RnOn",str(x),re.IGNORECASE)))
dogs["EvAwRnOn"] = dogs.groupby(by="Dog name")["EvAwRnOn"].cumsum()
dogs["QAwALdHldOn"] = dogs["Remarks"].apply(lambda x: len(re.findall("QAw,ALd,HldOn",str(x),re.IGNORECASE)))
dogs["QAwALdHldOn"] = dogs.groupby(by="Dog name")["QAwALdHldOn"].cumsum()
dogs["Crd1"] = dogs["Remarks"].apply(lambda x: len(re.findall("Crd1",str(x),re.IGNORECASE)))
dogs["Crd1"] = dogs.groupby(by="Dog name")["Crd1"].cumsum()
dogs["WEvCh"] = dogs["Remarks"].apply(lambda x: len(re.findall("W,EvCh",str(x),re.IGNORECASE)))
dogs["WEvCh"] = dogs.groupby(by="Dog name")["WEvCh"].cumsum()

grp= dogs.groupby(by=["Track","Trp","Dog name"])["won"]

dogs["Track number wins"]= convo_mean(dogs,  ["Track","Trp","Dog name"], "won")
dogs["Track number wins2"]= convo_mean(dogs,  ["Track","Trp"], "won")
#grp.transform(lambda x: x.expanding().mean())

dogs["Track number wins last 30"] = convo_mean(dogs, ["Track","Trp","Dog name"], "won",30)
dogs["Track number wins last 302"] = convo_mean(dogs, ["Track","Trp"], "won",30)
#grp.transform(lambda x: x.rolling(30,1).mean())

dogs["StdDev track number of wins"]=grp.transform(lambda x: x.expanding().std())

dogs["StdDev track number of wins last 100 races"]=grp.transform(lambda x: x.rolling(100,1).std())


grouped=races.groupby(by=pd.Grouper(key='Date_Time', axis=0, freq='D'))
end = time()
running_time = end - start
print('5time cost: %.5f sec' %running_time)


1 time cost: 0.45384 sec
2time cost: 15.60359 sec
3time cost: 30.23436 sec
4time cost: 36.26287 sec
5time cost: 74.05223 sec


In [9]:
dfs = [] 
for k, group in grouped:
    until = dogs.loc[dogs["Date_Time"].lt(k), ["A0","A1","A2","A3","A4","A5","A6","A7","Dog name", "Average SP","Average SP7","Average SP15","Average SP3","StdDevFin","StdDevFin7", "PlaceAverageFin","PlaceAverageFin7","PlaceAverageFin3","Total poss","Total win","Total win7","AverageBends","AverageBends_cum", "AverageBends_cum_first2", "AverageBends_cum_last2", "Total races", "Dogs profit", "AverageDistWonLoss", "AverageDistWonLoss2","Wins on this distance","Average position on this distance","Sire position on this distance","Dame position on this distance","Lowest CalTm","Lowest CalTm Av","Lowest CalTm Av Sire","Lowest CalTm Av Dame","Distance","Midcount", "EvCh","Bmp","Crd","SAw", "QAw", "Ld1","QAwALd","EPLd1","LdRnln","EPALd","ClrRn","EvAwRnOn","QAwALdHldOn","Crd1","WEvCh", "GradeWinsAverage", "GradeProfitability","Track number wins","Track number wins2", "Track number wins last 30","Track number wins last 302", "StdDev track number of wins", "StdDev track number of wins last 100 races"]].drop_duplicates("Dog name", keep="last")
    g = group.merge(until, on="Dog name", how="left",suffixes=(None,"_hist"))

    until_sire =dogs.loc[dogs["Date_Time"].lt(k),["Sire","SireProfitability"]].drop_duplicates("Sire", keep="last")
    g = g.merge(until_sire, on="Sire", how="left")

    until_dame = dogs.loc[dogs["Date_Time"].lt(k), ["Dame", "DameProfitability"]].drop_duplicates("Dame", keep="last")
    g = g.merge(until_dame, on="Dame", how="left")

    dfs.append(g)

df = pd.concat(dfs, axis=0, ignore_index=True)
df["Runners"] = df.groupby(by=["Raceid","Date_Time"])["Dog name"].transform(len)
trainer_cc_daily = df.groupby(by=["Trainer", pd.Grouper(key='Date_Time', axis=0, freq='D')])["Raceid"].cumcount()
trainer_cc = df.groupby(by=["Trainer"])["Raceid"].cumcount()
df["Total trainer races"]= trainer_cc-trainer_cc_daily
end = time()
running_time = end - start
print('6time cost: %.5f sec' %running_time)
df["Position"]=df["Position"].astype(int)
df["won"]=df["Position"].apply(lambda x: 1 if x==1 else 0 )
trainer_win = df.groupby(by=["Trainer"])["won"].cumsum()
trainer_win_daily=df.groupby(by=["Trainer", pd.Grouper(key='Date_Time', axis=0, freq='D')])["won"].cumsum()
df["Total trainer wins"]=trainer_win-trainer_win_daily

df["Last 20 trainer races"]=df.groupby("Trainer")["won"].transform(lambda x: x.rolling(20,1).sum())
c=df.groupby(by=[pd.Grouper(key='Date_Time', axis=0, freq='D'), "Trainer"])["Last 20 trainer races"].last()
c=c.reset_index()
c["Date_Time"]+=datetime.timedelta(days=1)
c.rename(columns={"Date_Time":"Date"}, inplace=True)
df.drop("Last 20 trainer races",axis=1,inplace=True)
df["Date"]=pd.to_datetime(df.Date_Time.dt.date)
df=df.merge(c,how="left",on=["Date","Trainer"])
end = time()
running_time = end - start
print('7time cost: %.5f sec' %running_time)
df["profit"]= df.apply(lambda x: x["Odds"] if x["won"] else -1,axis=1)
df["Trainer profit"]=df.groupby("Trainer")["profit"].transform(lambda x: x.expanding().sum())
df.drop("profit",inplace=True,axis=1)
c=df.groupby(by=[pd.Grouper(key='Date_Time', axis=0, freq='D'), "Trainer"])["Trainer profit"].last()
c=c.reset_index()
c["Date_Time"]+=datetime.timedelta(days=1)
c.rename(columns={"Date_Time":"Date"}, inplace=True)
df.drop("Trainer profit",axis=1,inplace=True)
df=df.merge(c,how="left",on=["Date","Trainer"])
end = time()
running_time = end - start
print('8time cost: %.5f sec' %running_time)


6time cost: 78.01703 sec
7time cost: 78.37311 sec
8time cost: 78.98334 sec


In [10]:
df["Distance same"]=df.apply(lambda x: 1 if x["Distance"]==x["Distance_hist"] else 0,axis=1)
df["Rank lowest CalTm"] = df.groupby("Raceid")["Lowest CalTm"].rank("min", ascending=True)
df["Rank Average bends"] = df.groupby("Raceid")["AverageBends_cum"].rank("min", ascending=True)
df["Rank Dog Profit"] = df.groupby("Raceid")["Dogs profit"].rank("min", ascending=True)
df["Rank Average position"] = df.groupby("Raceid")["Average position on this distance"].rank("min", ascending=True)
df["Rank Place_Average_fin7"] = df.groupby("Raceid")["PlaceAverageFin7"].rank("max", ascending=True)
df["AvgDistWonLossRank"] = df.groupby("Raceid")["AverageDistWonLoss2"].rank("max", ascending=True)
df["ClrRnRank"] = df.groupby("Raceid")["ClrRn"].rank("max", ascending=True)
df["QAwALdRank"] = df.groupby("Raceid")["QAwALd"].rank("max", ascending=True)
df["Born"]=pd.to_datetime(df["Born"], dayfirst=True)
df["Monthsold"]=(datetime.datetime.now()-df["Born"])/np.timedelta64(1, 'M')
df["Monthsold"]=df["Monthsold"].astype(int)




In [11]:
df["MonthsoldPossitionRatio"]=np.where(df["Average position on this distance"]>0,df["Monthsold"]/df["Average position on this distance"],0)
df["SawVsQaw"]=np.where(df["QAw"]>0,df["SAw"]/df["QAw"],0)
df["BmpVSQAw"]=np.where(df["QAw"]>0,df["Bmp"]/df["QAw"],0)
df["CrdVSBmp"]=np.where(df["Bmp"]>0,df["Crd"]/df["Bmp"],0)
df["SAwVSCrd"]=np.where(df["Crd"]>0,df["SAw"]/df["Crd"],0)
df["EvChVSBmp"]=np.where(df["Bmp"]>0,df["EvCh"]/df["Bmp"],0)
df["EvChVSCrd"]=np.where(df["Crd"]>0,df["EvCh"]/df["Crd"],0)
df["DameVsSireProf"]=np.where(df["DameProfitability"]>0,df["SireProfitability"]/df["DameProfitability"],0)
df["3vs15odds"]=np.where(df["Average SP15"]>0,df["Average SP3"]/df["Average SP15"],0)


df["MonthsoldPossitionRatioRank"] = df.groupby("Raceid")["MonthsoldPossitionRatio"].rank("min", ascending=True)
df["RankMonthsold"] = df.groupby("Raceid")["Monthsold"].rank("min", ascending=True)
end = time()
running_time = end - start
print('9time cost: %.5f sec' %running_time)
df.to_csv("output3.csv")
#train=df

9time cost: 79.36465 sec


In [12]:
#df["Test"]=df.apply(lambda x: 1 if x["Remarks"]=="QAw, ALd" else 0,axis=1)
df["TrainerRatio"]=np.where(df["Total trainer wins"]>0,df["Total trainer wins"]/df["Total trainer races"],0)
df["DogWinRatio"]=np.where(df["Total win"]>0,df["Total win"]/df["Total races"],0)
df["RankPlaceAverageFin3"] = df.groupby("Raceid")["PlaceAverageFin3"].rank("max", ascending=True)
train=df
print(len(df))
train.drop(train[train.Odds.eq(0)].index, inplace=True) #husk endret denne
print(len(train))
#train=train.fillna(0)
#train["y"]=y
#train.to_csv("del.csv")
#train.drop(train[train.Odds.eq("0")].index,inplace=True)

9686
9635


In [17]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split

gss = GroupKFold(n_splits=2)

X=train
X.dropna(subset=["Raceid","A0","A1","A2","A3","A4","A5","A6","A7"], inplace=True)


X=X.loc[:,["Raceid","A0","A1","A2","A3","A4","A5","A6","A7","BSP","Position","Number"]]

In [19]:
from tensorflow.keras.utils import to_categorical

X.drop(X.columns[X.isna().sum()>100], axis=1, inplace=True)
X.dropna(inplace=True)
X["Race_len"] = X.groupby(by="Raceid")["Raceid"].transform(len)
X.drop(X.index[X["Race_len"].ne(6)], axis=0, inplace=True)
df_pvt = X.pivot(index="Raceid", columns=["Number"],values=["A0","A1","A2","A3","A4","A5","A6","A7","BSP"])
X["Position"] = X["Position"].apply(lambda x: 1 if x==1 else 0)
group_winner = X.groupby(by="Raceid").apply(lambda x: x["Number"].at[x["Position"].idxmax()])
df_pvt.columns = [' '.join(map(str,col)).strip() for col in df_pvt.columns]
df_pvt = df_pvt.merge(pd.DataFrame(group_winner, columns=["winner"]), left_index=True, right_index=True)

y_true = to_categorical(df_pvt["winner"]-1)
y_true = np.concatenate([y_true,df_pvt.iloc[:,30:36].values],axis=1)

In [28]:
df_pvt

,A0 1,A0 2,A0 3,A0 4,A0 5,A0 6,A1 1,A1 2,A1 3,A1 4,A1 5,A1 6,A2 1,A2 2,A2 3,A2 4,A2 5,A2 6,A3 1,A3 2,A3 3,A3 4,A3 5,A3 6,A4 1,A4 2,A4 3,A4 4,A4 5,A4 6,A5 1,A5 2,A5 3,A5 4,A5 5,A5 6,A6 1,A6 2,A6 3,A6 4,A6 5,A6 6,A7 1,A7 2,A7 3,A7 4,A7 5,A7 6,BSP 1,BSP 2,BSP 3,BSP 4,BSP 5,BSP 6,winner
Raceid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
31179,-0.002834,-0.016247,-0.005558,-0.001508,-0.020609,-0.016376,0.069466,-0.002396,0.011434,-0.021834,0.011435,-0.073474,0.476505,0.267918,0.368754,1.520397,0.400677,-1.664435,-4.221627,-0.477047,-0.081515,0.018500,-0.189690,1.072848,3.521952,0.080676,-0.057696,-0.987474,-0.167994,-0.893885,0.290219,-0.181425,-0.270210,0.562545,0.002907,1.382538,0.209068,-0.095786,-0.076757,0.312966,-0.089191,-0.008062,0.705906,0.157746,0.082457,-4.367831,0.127469,0.400537,14.453053,5.781515,8.000000,7.503815,3.955368,4.182174,5
31180,-0.010936,-0.028436,-0.009918,-0.010987,-0.010445,-0.015771,-0.057374,-0.069832,0.011602,0.005894,-0.052225,-0.001825,-0.908644,0.851955,0.841721,0.293429,-0.618136,-0.881452,0.650191,0.108402,0.265144,-0.301803,0.911034,-0.250677,-1.111417,0.392049,-0.028366,0.037635,-0.904386,0.225544,0.436836,-0.556213,-0.525169,-0.216137,1.058673,0.254416,0.051658,-0.034095,-0.155200,-0.077796,0.151596,-0.038359,0.420035,-0.126932,-0.049494,0.144077,0.521644,-0.041223,5.771327,6.498276,14.402340,14.599592,6.212105,2.605166,1
31181,-0.005213,-0.008077,-0.001098,-0.005213,-0.011145,-0.017459,-0.025672,0.067638,0.012490,-0.025672,0.002275,0.061807,2.314214,-5.559526,0.295732,2.314214,-0.797308,-6.216420,0.160035,1.557535,-0.033698,0.160035,-0.238047,1.254398,-1.670802,0.558652,0.007494,-1.670802,0.217555,0.789407,-5.511973,-1.682297,-0.005563,-5.511973,0.086100,-1.387690,-1.857773,-0.268740,0.024495,-1.857773,-0.051204,-0.662194,1.542704,-0.829075,-0.141379,1.542704,0.084405,-0.640730,11.985853,2.704662,12.583670,3.400000,7.584570,21.251381,2
31182,-0.020250,-0.023504,-0.001098,-0.009676,-0.015180,-0.014990,-0.060802,-0.058782,0.012490,0.012576,0.016976,-0.052513,-0.621691,0.895861,0.295732,0.905421,0.476002,-0.596180,0.266346,0.151470,-0.033698,0.355582,0.030597,0.441590,-1.014233,-1.082548,0.007494,0.268352,-0.263325,-1.057274,0.748073,0.572215,-0.005563,-0.431051,-0.051166,0.713361,-0.038392,0.040160,0.024495,-0.146638,-0.088153,-0.020401,0.415049,0.591908,-0.141379,-0.169842,0.065848,0.401381,9.155532,4.474477,9.659049,3.752761,4.349596,14.271051,1
31183,-0.002834,-0.017709,-0.015984,-0.005715,-0.005715,-0.006010,0.069466,-0.023164,0.015535,0.007815,0.007815,0.011040,0.476505,-1.784974,0.484821,-0.721983,-0.721983,0.018878,-4.221627,-0.238847,-0.177060,-0.017760,-0.017760,-0.250861,3.521952,0.263108,-0.175982,0.122224,0.122224,0.172799,0.290219,0.276360,-0.165409,0.032027,0.032027,0.122481,0.209068,-0.241698,-0.102036,-0.050166,-0.050166,0.017449,0.705906,0.102594,0.253096,0.022784,0.022784,-0.118312,6.200000,2.152150,6.358586,8.510038,30.000000,18.349649,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33400,-0.013901,-0.010818,-0.020366,-0.011855,-0.015771,0.008135,-0.049537,0.003145,-0.001254,-0.024462,-0.001825,0.152973,0.705454,0.343243,0.804586,-0.534644,-0.881452,-0.579004,0.215043,-0.256759,-0.039949,0.080469,-0.250677,0.737418,-0.955373,-0.014655,0.406724,-0.552217,0.225544,-0.191236,0.742516,-0.235498,-0.342267,0.524615,0.254416,-0.372852,0.122559,-0.094748,-0.165667,0.126019,-0.038359,-0.085630,0.455147,0.096125,-0.094553,0.376647,-0.041223,-0.064092,3.550000,7.400000,4.100000,4.262721,12.050055,32.000000,2
33401,-0.014937,-0.015602,-0.016405,-0.006814,-0.008641,-0.001385,0.004287,-0.004574,-0.006015,0.009599,-0.041247,0.015139,0.879911,-0.831638,-0.822819,0.027697,0.730964,0.103022,0.180339,-0.205634,-0.413291,-0.458518,0.390287,-0.238230,0.311393,0.173253,0.260596,0.260141,-0.998414,0.164810,-0.39

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_pvt.iloc[:,:6].values, y_true, test_size=0.33, random_state=42)




In [37]:
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras.optimizers import RMSprop
from keras import regularizers, Input
from keras.callbacks import Callback
from keras.models import load_model
from keras.metrics import accuracy
from keras.models import Sequential
from keras import backend as K
import pandas as pd

inputs = Input(shape=(X_train.shape[1],), name="digits")
outputs = Dense(6, activation="softmax", name="predictions")(inputs)

model = Model(inputs=inputs, outputs=outputs)


def profitability_softmax(y_true, y_pred):
    y_true_and_w = y_true
    y_true = y_true_and_w[:, :6]
    odds = y_true_and_w[:, 6:]
    return K.sum(y_true*odds -1) * K.round(K.sigmoid(odds*y_pred-1))

def odds_loss_softmax(y_true, y_pred):
    y_true_and_w = y_true
    y_true = y_true_and_w[:, :6]
    odds = y_true_and_w[:, 6:]
    return -K.sum((y_true * odds - 1) * K.relu(y_pred * odds-1))

opt = RMSprop(learning_rate=0.001)
model.compile(loss=odds_loss_softmax, optimizer=opt, metrics=[profitability_softmax])


In [41]:
model.fit(X_train, y_train, epochs=20,batch_size=32)

Epoch 1/20
28/28 [==============================] - 14s 1ms/step - loss: 0.1544 - profitability_softmax: -0.1735    
Epoch 2/20
28/28 [==============================] - 0s 1ms/step - loss: 0.1540 - profitability_softmax: -0.1633   
Epoch 3/20
28/28 [==============================] - 0s 1ms/step - loss: 0.0995 - profitability_softmax: -0.1094   
Epoch 4/20
28/28 [==============================] - 0s 1ms/step - loss: 0.0866 - profitability_softmax: -0.0926   
Epoch 5/20
28/28 [==============================] - 0s 1ms/step - loss: 0.2565 - profitability_softmax: -0.2848
Epoch 6/20
28/28 [==============================] - 0s 1ms/step - loss: 0.1117 - profitability_softmax: -0.1336   
Epoch 7/20
28/28 [==============================] - 0s 1ms/step - loss: 0.1935 - profitability_softmax: -0.2273   
Epoch 8/20
28/28 [==============================] - 0s 1ms/step - loss: 0.1460 - profitability_softmax: -0.1524
Epoch 9/20
28/28 [==============================] - 0s 1ms/step - loss: 0.1871 - pro

In [42]:
results = model.evaluate(X_test, y_test)

14/14 [==============================] - 0s 1ms/step - loss: 0.3316 - profitability_softmax: -0.1217   


In [43]:
model.weights

[<tf.Variable 'predictions/kernel:0' shape=(6, 6) dtype=float32, numpy=
 array([[-0.39947402,  0.08375883,  0.6338906 ,  0.4529006 , -0.42449686,
          0.09340358],
        [ 0.05939442, -0.06522486, -0.11315361, -0.41507176, -0.2980746 ,
         -0.17344102],
        [-0.42566895,  0.08185693, -0.26846176, -0.3423915 , -0.03307693,
          0.15181583],
        [ 0.47872394,  0.50745285,  0.6604475 , -0.47044083, -0.60139126,
         -0.39244035],
        [-0.20886579, -0.15454657,  0.35216203,  0.68661875,  0.06576507,
         -0.36448476],
        [-0.39105436, -0.5157182 , -0.40326843,  0.29066208,  0.2837554 ,
         -0.5326643 ]], dtype=float32)>,
 <tf.Variable 'predictions/bias:0' shape=(6,) dtype=float32, numpy=
 array([0.01559898, 0.16990125, 0.01187951, 0.01481845, 0.00966483,
        0.1699059 ], dtype=float32)>]